## Serving Function

This notebook demonstrates the Serving Function design pattern using Keras


## Simple text classification model

This model uses transfer learning with TensorFlow Hub and Keras. It is based on https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

It classifies movie reviews as positive or negative using the text of the review. The reviews come from an IMDB dataset that contains the text of 50,000 movie reviews from the Internet Movie Database. These are split into 25,000 reviews for training and 25,000 reviews for testing.

In [1]:
# Already installed if you are using Cloud AI Platform Notebooks
#!pip install -q tensorflow-hub
#!pip install -q tfds-nightly

In [2]:
import numpy as np
import tensorflow as tf


import tensorflow_hub as hub
import tensorflow_datasets as tfds
train_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train', 'test'),
    as_supervised=True)

In [3]:
split = 3 # 1/4 records is validation
dataset_train = train_data.window(split, split + 1).flat_map(lambda *ds: ds[0] if len(ds) == 1 else tf.data.Dataset.zip(ds))
dataset_validation = train_data.skip(split).window(1, split + 1).flat_map(lambda *ds: ds[0] if len(ds) == 1 else tf.data.Dataset.zip(ds))

In [4]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True, name='full_text')

In [5]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu', name='h1_dense'))
model.add(tf.keras.layers.Dense(1, name='positive_review_logits'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
full_text (KerasLayer)       (None, 20)                389380    
_________________________________________________________________
h1_dense (Dense)             (None, 16)                336       
_________________________________________________________________
positive_review_logits (Dens (None, 1)                 17        
Total params: 389,733
Trainable params: 389,733
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])



history = model.fit(dataset_train.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=dataset_validation.batch(512),
                    verbose=1)

Epoch 1/10
37/37 [==============================] - 12s 333ms/step - loss: 0.7282 - accuracy: 0.6085 - val_loss: 0.6422 - val_accuracy: 0.6398
Epoch 2/10
37/37 [==============================] - 11s 292ms/step - loss: 0.6184 - accuracy: 0.6599 - val_loss: 0.5946 - val_accuracy: 0.6651
Epoch 3/10
37/37 [==============================] - 11s 298ms/step - loss: 0.5738 - accuracy: 0.6869 - val_loss: 0.5533 - val_accuracy: 0.7013
Epoch 4/10
37/37 [==============================] - 11s 296ms/step - loss: 0.5307 - accuracy: 0.7202 - val_loss: 0.5143 - val_accuracy: 0.7320
Epoch 5/10
37/37 [==============================] - 11s 295ms/step - loss: 0.4898 - accuracy: 0.7511 - val_loss: 0.4803 - val_accuracy: 0.7664
Epoch 6/10
37/37 [==============================] - 11s 299ms/step - loss: 0.4480 - accuracy: 0.7796 - val_loss: 0.4454 - val_accuracy: 0.7800
Epoch 7/10
37/37 [==============================] - 11s 295ms/step - loss: 0.4092 - accuracy: 0.8059 - val_loss: 0.4162 - val_accuracy: 0.8042

In [7]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.359
accuracy: 0.834


## Export the model for serving

model.save() writes out a "serve" tag_set

In [8]:
import os, datetime, shutil
shutil.rmtree('export/default', ignore_errors=True)
export_path = os.path.join('export', 'default', 'sentiment_{}'.format(datetime.datetime.now().strftime("%Y%m%d_%H%M%S")))
model.save(export_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: export/default/sentiment_20200505_184058/assets


INFO:tensorflow:Assets written to: export/default/sentiment_20200505_184058/assets


In [9]:
!find export/default

export/default
export/default/sentiment_20200505_184058
export/default/sentiment_20200505_184058/variables
export/default/sentiment_20200505_184058/variables/variables.data-00000-of-00001
export/default/sentiment_20200505_184058/variables/variables.index
export/default/sentiment_20200505_184058/assets
export/default/sentiment_20200505_184058/assets/tokens.txt
export/default/sentiment_20200505_184058/saved_model.pb


In [11]:
!saved_model_cli show --dir {export_path} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['full_text_input'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_full_text_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['positive_review_logits'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall_2:0
Method name is: tensorflow/serving/predict


In [62]:
## illustrates how we can load this model and do inference based on the signature above
restored = tf.keras.models.load_model(export_path)
review1 = 'The film is based on a prize-winning novel.' # neutral
review2 = 'The film is fast moving and has several great action scenes.' # positive
review3 = 'The film was very boring. I walked out half-way.' # negative

infer = restored.signatures['serving_default']
outputs = infer(full_text_input=tf.constant([review1, review2, review3])) # note input name
logit = outputs['positive_review_logits']  # note output name
print(1 / (1 + np.exp(-logit))) # probability

[[0.46924272]
 [0.85492635]
 [0.15715358]]


## Custom serving function

Let's write out a new signature. But this time, let's carry out the sigmoid operation, so that the model outputs a probability.

In [63]:
@tf.function(input_signature=[tf.TensorSpec([None], dtype=tf.string)])
def add_prob(reviews):
    logits = model(reviews, training=False) # the model is captured via closure
    probs = tf.sigmoid(logits)
    return {
        'positive_review_logits' : logits,
        'positive_review_probability' : probs
    }
shutil.rmtree('export/probs', ignore_errors=True)
probs_export_path = os.path.join('export', 'probs', 'sentiment_{}'.format(datetime.datetime.now().strftime("%Y%m%d_%H%M%S")))
model.save(probs_export_path, signatures={'serving_default': add_prob})

INFO:tensorflow:Assets written to: export/probs/sentiment_20200505_192836/assets


INFO:tensorflow:Assets written to: export/probs/sentiment_20200505_192836/assets


In [64]:
!saved_model_cli show --dir {probs_export_path} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['reviews'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_reviews:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['positive_review_logits'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall_2:0
  outputs['positive_review_probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall_2:1
Method name is: tensorflow/serving/predict


In [66]:
restored = tf.keras.models.load_model(probs_export_path)
infer = restored.signatures['serving_default']
outputs = infer(reviews=tf.constant([review1, review2, review3])) # note input name
probs = outputs['positive_review_probability']  # note output name
print(probs)

tf.Tensor(
[[0.46924272]
 [0.85492635]
 [0.15715358]], shape=(3, 1), dtype=float32)


## Deploy to Cloud AI Platform Predictions

We can deploy the model to AI Platform Predictions which will take care of scaling

In [76]:
!find export/probs | head -2 | tail -1

export/probs/sentiment_20200505_192836


In [78]:
%%bash

MODEL_LOCATION=$(find export/probs | head -2 | tail -1)
MODEL_NAME=imdb
MODEL_VERSION=v1

TFVERSION=2.1
REGION=us-central1
BUCKET=ai-analytics-solutions-kfpdemo

# create the model if it doesn't already exist
modelname=$(gcloud ai-platform models list | grep -w "$MODEL_NAME")
echo $modelname
if [ -z "$modelname" ]; then
   echo "Creating model $MODEL_NAME"
   gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
else
   echo "Model $MODEL_NAME already exists"
fi

# delete the model version if it already exists
modelver=$(gcloud ai-platform versions list --model "$MODEL_NAME" | grep -w "$MODEL_VERSION")
echo $modelver
if [ "$modelver" ]; then
   echo "Deleting version $MODEL_VERSION"
   yes | gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
   sleep 10
fi


echo "Creating version $MODEL_VERSION from $MODEL_LOCATION"
gcloud ai-platform versions create ${MODEL_VERSION} \
       --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --staging-bucket gs://${BUCKET} \
       --runtime-version $TFVERSION

imdb
Model imdb already exists

Creating version v1 from export/probs/sentiment_20200505_192836


Listed 0 items.
Creating version (this might take a few minutes)......
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................done.


In [79]:
%%writefile input.json
{"reviews": "The film is based on a prize-winning novel."}
{"reviews": "The film is fast moving and has several great action scenes."}
{"reviews": "The film was very boring. I walked out half-way."}

Writing input.json


In [80]:
!gcloud ai-platform predict --model imdb --json-instances input.json --version v1

POSITIVE_REVIEW_LOGITS  POSITIVE_REVIEW_PROBABILITY
[-0.12318471074104309]  [0.469242662191391]
[1.773773431777954]     [0.854926347732544]
[-1.6795613765716553]   [0.15715356171131134]


To take a quick anonymous survey, run:
  $ gcloud survey



In [81]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build("ml", "v1", credentials = credentials,
            discoveryServiceUrl = "https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json")

request_data = {"instances":
  [
      {"reviews": "The film is based on a prize-winning novel."},
      {"reviews": "The film is fast moving and has several great action scenes."},
      {"reviews": "The film was very boring. I walked out half-way."}
  ]
}

parent = "projects/{}/models/imdb".format("ai-analytics-solutions", "v1") # use default version

response = api.projects().predict(body = request_data, name = parent).execute()
print("response = {0}".format(response))

response = {'predictions': [{'positive_review_probability': [0.469242662191391], 'positive_review_logits': [-0.12318471074104309]}, {'positive_review_probability': [0.854926347732544], 'positive_review_logits': [1.773773431777954]}, {'positive_review_probability': [0.15715356171131134], 'positive_review_logits': [-1.6795613765716553]}]}


In [83]:
print(response['predictions'][0]['positive_review_probability'][0])

0.469242662191391


Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License